# LABORATORIO #1 
 Autores: Alfredo Quezada
 
 Autores: Randy Venegas

### Parte 1

* 1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones. 

In [1]:
import pandas as pd

df = pd.read_csv('dataset_pishing.csv')
df.sample(n=5).head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
4111,http://patents.justia.com/assignee/input-outpu...,0,0,0,0,0,0,0,0,0,...,0,0,0,3350,6512,3819,0,0,5,legitimate
257,http://signin.eday.co.uk.ws.edayisapi.dllsign....,0,0,0,0,0,0,0,0,0,...,1,0,0,300,65,0,0,1,0,phishing
10821,https://www.soundsnap.com/tags/modem,0,1,0,0,0,0,0,0,0,...,0,0,0,1084,5125,18864,0,0,5,legitimate
6112,https://iuj24.app.link/jciozKXoB8,0,0,0,0,0,0,0,0,0,...,1,0,0,2416,1967,4657,0,1,3,phishing
3181,https://r7vfe.csb.app/,0,0,0,0,0,0,0,0,0,...,1,0,0,290,806,45133,0,1,1,phishing


* 2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como 
“pishing”. ¿Está balanceado el dataset? 

In [2]:
print(df['status'].value_counts())

legitimate    5715
phishing      5715
Name: status, dtype: int64


        El modelo, como se puede observar si se encuentra balanceado. 

* 3. Para ello escriba las funciones necesarias y genere las nuevas columnas del dataset. Muestre un nuevo 
ejemplo de cinco observaciones donde se visualicen algunas de las columnas nuevas. 